In [ ]:
#import packages
from statsmodels.tsa.api import VAR
import pandas as pd
import matplotlib.pyplot as plt
from time_series_validation import load_states
import numpy as np

In [ ]:
#defining the states in New England. Make a data frame with just the New England states, indexed by date

covid_states, state_list = load_states()
new_england = ["MA","ME","NH","VT","RI","CT"]
df = pd.DataFrame(np.array([covid_states[state].beds for state in new_england]).transpose(),
            columns=new_england,
                 index=covid_states["MA"].date)
# Get the number of new hospital beds used per day by subtracting a day's totals from the previous day's totals
# Get rid of data from after 8/31/21 to keep as test data for later on
df_diff = df.copy()
for state in new_england:
    df_diff[state] = df[state].diff(1)
df_diff = df_diff.iloc[1:].copy()
df_diff = df_diff.iloc[df_diff.index < "2021-09-01"].copy()

In [ ]:
lag_order = 4 # This is the number of previous days the model is predicting on.
# Use days n-4, n-3, n-2, n-1 to predict day n, then use days n-3, n-2, n-1, n (where day n comes from last prediction)
#to predict day n+1...keep going until 31 days into the future
future = 31
model = VAR(df_diff.values)
results = model.fit(lag_order)
predictions = results.forecast(df_diff.values[-lag_order:], future)

In [ ]:
for state_num in range(len(new_england)):
    state_info = df[new_england[state_num]].loc[df.index < "2021-10-01"]

    # Un-difference the sequence
    predicted = np.cumsum(predictions[:,state_num]) + state_info.loc["2021-08-31"]
    
    num_actual = len(state_info)
    plt.figure(figsize=(8,8))
    plt.plot(range(num_actual),state_info, label="actual")
    plt.plot(range(len(df_diff), len(df_diff)+future), predicted, label="predicted")
    plt.legend()
    plt.title(new_england[state_num])
    plt.show()

In [ ]:
print(predictions[:,5])